In [3]:
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("EDA Proyecto1") \
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate()

spark.catalog.setCurrentDatabase("proyecto1db")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1748300572937_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark.catalog.listTables()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Table(name='tblpmo_data', catalog='spark_catalog', namespace=['proyecto1db'], description=None, tableType='EXTERNAL', isTemporary=False)]

Total de filas

In [5]:
spark.sql("SELECT COUNT(*) AS total_rows FROM tblpmo_data").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|total_rows|
+----------+
|      2117|
+----------+

In [7]:
spark.sql("SELECT * FROM tblpmo_data LIMIT 10")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   c0                            project_id  ... change_score num_tasks
0   0  0195AB40-ABFB-ED11-8848-000D3A5A3338  ...          0.0      18.0
1   1  0685B55C-2CE8-ED11-A7C6-000D3A31C831  ...          0.0      20.0
2   2  09CBA925-444F-EF11-A317-6045BD09C5C1  ...          0.0      22.0
3   3  0C5C42BB-19BA-4E3C-B998-D956EFC2420B  ...          0.0       3.0
4   4  11C4A42F-DB03-4D0B-9F90-ECD0B7E6A262  ...          0.0      28.0
5   7  20364FF9-88ED-414B-9E88-6BC03946B820  ...          0.0      29.0
6   8  21B5DF1C-D7C3-ED11-83FF-000D3A31C831  ...          0.0      19.0
7   9  21F352BC-8620-EE11-9CBE-000D3A5A3338  ...          0.0      17.0
8  10  27CCEA0A-9149-4486-9A17-0D86567C05D9  ...          0.0      10.0
9  11  2AAA92F2-504E-EF11-A317-6045BD09C5C1  ...          0.0      35.0

[10 rows x 40 columns]

Verificamos que el total de nulos en las columnas de project_id,project_name,effort,project_priority, avg_risk_exposure, total_headcount, num_tasks y duration sea 0 (se manejaron estos casos en el dataprep)

In [17]:
spark.sql("""SELECT
  SUM(CASE WHEN project_id IS NULL THEN 1 ELSE 0 END) AS null_project_id,
  SUM(CASE WHEN project_name IS NULL THEN 1 ELSE 0 END) AS null_project_name,
  SUM(CASE WHEN effort IS NULL THEN 1 ELSE 0 END) AS null_effort,
  SUM(CASE WHEN project_priority IS NULL THEN 1 ELSE 0 END) AS null_project_priority,
  SUM(CASE WHEN avg_risk_exposure IS NULL THEN 1 ELSE 0 END) AS avg_risk_exposure,
  SUM(CASE WHEN total_headcount IS NULL THEN 1 ELSE 0 END) AS total_headcount,
  SUM(CASE WHEN num_tasks IS NULL THEN 1 ELSE 0 END) AS num_tasks,
  SUM(CASE WHEN duration IS NULL THEN 1 ELSE 0 END) AS duration
FROM tblpmo_data;""").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----------------+-----------+---------------------+-----------------+---------------+---------+--------+
|null_project_id|null_project_name|null_effort|null_project_priority|avg_risk_exposure|total_headcount|num_tasks|duration|
+---------------+-----------------+-----------+---------------------+-----------------+---------------+---------+--------+
|              0|                0|          0|                    0|                0|              0|        0|       0|
+---------------+-----------------+-----------+---------------------+-----------------+---------------+---------+--------+

Agrupamos por prioridad

In [7]:
spark.sql("""SELECT project_priority, COUNT(*) AS count
FROM tblpmo_data
GROUP BY project_priority
ORDER BY count DESC;""").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+
|project_priority|count|
+----------------+-----+
|               3| 1989|
|               2|   88|
|               4|   24|
|               1|   16|
+----------------+-----+

Agrupamos por unidad de negocio, y calculamos el esfuerzo, duración, exposición a riesgo y número de tareas

In [6]:
bu_df = spark.sql("""SELECT business_unit, AVG(effort) AS avg_effort, AVG(duration) AS avg_duration, AVG(avg_risk_exposure) AS avg_risk_exposure, AVG(num_tasks) AS avg_num_tasks
FROM tblpmo_data
GROUP BY business_unit
ORDER BY avg_effort DESC;""")

s3_output_path = "s3://refined-proyecto-integrador/eda/business_unit"
bu_df.write.mode("overwrite").parquet(s3_output_path)

bu_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+------------------+-----------------+------------------+-----------------+
|business_unit|        avg_effort|     avg_duration| avg_risk_exposure|    avg_num_tasks|
+-------------+------------------+-----------------+------------------+-----------------+
|       IT PMO|2030.4398241206034|95.44522613065325|2.7106532663316574|68.21608040201005|
|   Global PMO|1256.6474578243165| 97.3410413030832| 2.018586387434558|62.56544502617801|
+-------------+------------------+-----------------+------------------+-----------------+

In [22]:
spark.sql("""SELECT
  AVG(effort) AS avg_effort, AVG(duration) AS avg_duration, AVG(avg_risk_exposure) AS avg_risk_exposure, AVG(num_tasks) AS avg_num_tasks
FROM tblpmo_data;""").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------------+------------------+-----------------+
|        avg_effort|     avg_duration| avg_risk_exposure|    avg_num_tasks|
+------------------+-----------------+------------------+-----------------+
|1402.1218847425598|96.98462446858764|2.1486962683042035|63.62777515351913|
+------------------+-----------------+------------------+-----------------+

Agrupamos por salud del proyecto (si está)

In [9]:
spark.sql("""SELECT overall_health, COUNT(*) AS count
FROM tblpmo_data
GROUP BY overall_health
ORDER BY count DESC;""").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-----+
|overall_health|count|
+--------------+-----+
|  (1) On Track| 2000|
|   (2) At Risk|   75|
| (3) Off Track|   40|
|          NULL|    2|
+--------------+-----+

Tendencia del proyecto

In [10]:
spark.sql("""SELECT project_trend, COUNT(*) AS count
FROM tblpmo_data
GROUP BY project_trend
ORDER BY count DESC;
;""").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----+
|project_trend|count|
+-------------+-----+
|    Unchanged| 1477|
|    Improving|  539|
|         NULL|   75|
|Deteriorating|   26|
+-------------+-----+

In [11]:
spark.sql("""SELECT country, COUNT(*) AS project_count
FROM tblpmo_data
GROUP BY country
ORDER BY project_count DESC
LIMIT 10;""").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------+
|             country|project_count|
+--------------------+-------------+
|         Philippines|          381|
|               India|          273|
|               Egypt|          105|
|              Global|          100|
|United States of ...|           99|
|            Colombia|           99|
|              Brazil|           94|
|            Malaysia|           75|
|               China|           58|
|        South Africa|           50|
+--------------------+-------------+

Clientes más comunes

In [14]:
spark.sql("""SELECT
  CASE WHEN client IS NULL THEN 'Anonymous' ELSE client END AS client_name,
  COUNT(*) AS count
FROM tblpmo_data
GROUP BY client_name
ORDER BY count DESC;""").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----+
|    client_name|count|
+---------------+-----+
|      Anonymous|  796|
|         Amazon|   39|
|      ByteDance|   29|
|         Google|   25|
|         Avalon|   23|
|           TEMU|   14|
|           Uber|   12|
|           VMO2|   12|
|        Expedia|   12|
|          Agoda|   10|
|           DISH|   10|
|         Intuit|    9|
|      Microsoft|    9|
|            UHG|    9|
|         Airbnb|    9|
|Puntos Colombia|    9|
|         Disney|    8|
|           Meta|    8|
|    Booking.com|    8|
|        avianca|    8|
+---------------+-----+
only showing top 20 rows